##**4. LSTM, GRU**
1. 기존 RNN과 다른 부분에 대해서 배웁니다.
2. 이전 실습에 이어 다양한 적용법을 배웁니다.

### **필요 패키지 import**

In [1]:
from tqdm import tqdm
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torch

### **데이터 전처리**

아래의 sample data를 확인해봅시다.  
이전 실습과 동일합니다.

In [2]:
vocab_size = 100
pad_id = 0

data = [
  [85,14,80,34,99,20,31,65,53,86,3,58,30,4,11,6,50,71,74,13],
  [62,76,79,66,32],
  [93,77,16,67,46,74,24,70],
  [19,83,88,22,57,40,75,82,4,46],
  [70,28,30,24,76,84,92,76,77,51,7,20,82,94,57],
  [58,13,40,61,88,18,92,89,8,14,61,67,49,59,45,12,47,5],
  [22,5,21,84,39,6,9,84,36,59,32,30,69,70,82,56,1],
  [94,21,79,24,3,86],
  [80,80,33,63,34,63],
  [87,32,79,65,2,96,43,80,85,20,41,52,95,50,35,96,24,80]
]

In [3]:
max_len = len(max(data, key=len))
print(f"Maximum sequence length: {max_len}")

valid_lens = []
for i, seq in enumerate(tqdm(data)):
  valid_lens.append(len(seq))
  if len(seq) < max_len:
    data[i] = seq + [pad_id] * (max_len - len(seq))

Maximum sequence length: 20


100%|█████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 163840.00it/s]


In [4]:
# B: batch size, L: maximum sequence length
batch = torch.LongTensor(data)  # (B, L)
batch_lens = torch.LongTensor(valid_lens)  # (B)

batch_lens, sorted_idx = batch_lens.sort(descending=True)
batch = batch[sorted_idx]

print(batch)
print(batch_lens)

tensor([[85, 14, 80, 34, 99, 20, 31, 65, 53, 86,  3, 58, 30,  4, 11,  6, 50, 71,
         74, 13],
        [58, 13, 40, 61, 88, 18, 92, 89,  8, 14, 61, 67, 49, 59, 45, 12, 47,  5,
          0,  0],
        [87, 32, 79, 65,  2, 96, 43, 80, 85, 20, 41, 52, 95, 50, 35, 96, 24, 80,
          0,  0],
        [22,  5, 21, 84, 39,  6,  9, 84, 36, 59, 32, 30, 69, 70, 82, 56,  1,  0,
          0,  0],
        [70, 28, 30, 24, 76, 84, 92, 76, 77, 51,  7, 20, 82, 94, 57,  0,  0,  0,
          0,  0],
        [19, 83, 88, 22, 57, 40, 75, 82,  4, 46,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [93, 77, 16, 67, 46, 74, 24, 70,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [94, 21, 79, 24,  3, 86,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [80, 80, 33, 63, 34, 63,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 76, 79, 66, 32,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0]])
tensor([2

### **LSTM 사용**

LSTM에선 cell state가 추가됩니다.  
Cell state의 shape는 hidden state의 그것과 동일합니다.

In [5]:
embedding_size = 256
hidden_size = 512
num_layers = 1
num_dirs = 1

embedding = nn.Embedding(vocab_size, embedding_size)
lstm = nn.LSTM(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)
c_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h)

In [12]:
h_0

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [14]:
h_0.shape

torch.Size([1, 10, 512])

In [16]:
c_0 == h_0

tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])

In [7]:
# d_w: word embedding size
batch_emb = embedding(batch)  # (B, L, d_w)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, (h_n, c_n) = lstm(packed_batch, (h_0, c_0))

In [8]:
packed_outputs

PackedSequence(data=tensor([[ 0.1288,  0.0810,  0.0717,  ...,  0.0009,  0.0555, -0.1259],
        [ 0.0678, -0.0109,  0.0860,  ..., -0.0908, -0.0636,  0.0780],
        [-0.0351,  0.1051, -0.0605,  ...,  0.0954,  0.0735,  0.0010],
        ...,
        [ 0.1678, -0.0224,  0.1283,  ...,  0.1792,  0.1592,  0.1147],
        [-0.1584,  0.1448,  0.1870,  ..., -0.1245,  0.0481,  0.1308],
        [ 0.0462, -0.0336,  0.2350,  ..., -0.0415,  0.1065, -0.0899]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)

In [9]:
packed_outputs[0].shape # (패딩 없는 실제 데이터 총 길이(총 단어수), h_t)

torch.Size([123, 512])

In [10]:
h_n.shape

torch.Size([1, 10, 512])

In [11]:
c_n.shape

torch.Size([1, 10, 512])

In [ ]:
outputs, output_lens = pad_packed_sequence(packed_outputs)
print(outputs.shape)
print(output_lens)

### **GRU 사용**

GRU는 cell state가 없어 RNN과 동일하게 사용 가능합니다.   
GRU를 이용하여 LM task를 수행해봅시다.

In [17]:
gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=True if num_dirs > 1 else False
)

In [18]:
output_layer = nn.Linear(hidden_size, vocab_size)

In [19]:
input_id = batch.transpose(0, 1)[0, :]  # (B)
hidden = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (1, B, d_h)

Teacher forcing 없이 이전에 얻은 결과를 다음 input으로 이용합니다.

In [20]:
for t in range(max_len):
  input_emb = embedding(input_id).unsqueeze(0)  # (1, B, d_w)
  output, hidden = gru(input_emb, hidden)  # output: (1, B, d_h), hidden: (1, B, d_h)

  # V: vocab size
  output = output_layer(output)  # (1, B, V)
  probs, top_id = torch.max(output, dim=-1)  # probs: (1, B), top_id: (1, B)

  print("*" * 50)
  print(f"Time step: {t}")
  print(output.shape)
  print(probs.shape)
  print(top_id.shape)

  input_id = top_id.squeeze(0)  # (B)

**************************************************
Time step: 0
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 1
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 2
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 3
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 4
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 5
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 6
torch.Size([1, 10, 100])
torch.Size([1, 10])
torch.Size([1, 10])
**************************************************
Time step: 7
torch.Size([1, 10, 100])
torch.Si

`max_len`만큼의 for 문을 돌면서 모든 결과물의 모양을 확인했지만 만약 종료 조건(예를 들어 문장의 끝을 나타내는 end token 등)이 되면 중간에 생성을 그만둘 수도 있습니다.

### **양방향 및 여러 layer 사용**

이번엔 양방향 + 2개 이상의 layer를 쓸 때 얻을 수 있는 결과에 대해 알아봅니다.

In [21]:
num_layers = 2
num_dirs = 2
dropout=0.1

gru = nn.GRU(
    input_size=embedding_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    dropout=dropout,
    bidirectional=True if num_dirs > 1 else False
)

Bidirectional이 되었고 layer의 개수가 $2$로 늘었기 때문에 hidden state의 shape도 `(4, B, d_h)`가 됩니다.

In [23]:
# d_w: word embedding size, num_layers: layer의 개수, num_dirs: 방향의 개수
batch_emb = embedding(batch)  # (B, L, d_w)
h_0 = torch.zeros((num_layers * num_dirs, batch.shape[0], hidden_size))  # (num_layers * num_dirs, B, d_h) = (4, B, d_h)

packed_batch = pack_padded_sequence(batch_emb.transpose(0, 1), batch_lens)

packed_outputs, h_n = gru(packed_batch, h_0)

In [24]:
packed_outputs

PackedSequence(data=tensor([[ 0.0298,  0.0039, -0.0505,  ...,  0.0174, -0.0399,  0.1782],
        [ 0.0672,  0.0723,  0.1700,  ...,  0.1121,  0.0232,  0.0243],
        [-0.1066,  0.1780, -0.0096,  ...,  0.0701,  0.0344,  0.0226],
        ...,
        [ 0.1464, -0.0696,  0.0491,  ..., -0.1093,  0.0057, -0.0259],
        [ 0.1276, -0.1609, -0.1486,  ...,  0.0088, -0.0297, -0.0266],
        [ 0.1326, -0.2341, -0.1868,  ...,  0.0279, -0.0313, -0.1149]],
       grad_fn=<CatBackward0>), batch_sizes=tensor([10, 10, 10, 10, 10,  9,  7,  7,  6,  6,  5,  5,  5,  5,  5,  4,  4,  3,
         1,  1]), sorted_indices=None, unsorted_indices=None)

In [25]:
packed_outputs[0].shape

torch.Size([123, 1024])

In [26]:
h_n.shape

torch.Size([4, 10, 512])

In [27]:
outputs, output_lens = pad_packed_sequence(packed_outputs)

print(outputs.shape)  # (L, B, num_dirs*d_h)
print(output_lens)

torch.Size([20, 10, 1024])
tensor([20, 18, 18, 17, 15, 10,  8,  6,  6,  5])


각각의 결과물의 shape는 다음과 같습니다.

`outputs`: `(max_len, batch_size, num_dir * hidden_size)`  
`h_n`: `(num_layers*num_dirs, batch_size, hidden_size)`

In [29]:
batch_size = h_n.shape[1]

In [30]:
h_n.view(num_layers, num_dirs, batch_size, hidden_size)

tensor([[[[ 0.4195,  0.1477,  0.0096,  ..., -0.1743,  0.1848, -0.0854],
          [-0.1352, -0.2079, -0.0161,  ...,  0.1982,  0.1403,  0.2232],
          [-0.2249,  0.2340,  0.5877,  ..., -0.0396, -0.1890, -0.4129],
          ...,
          [-0.1610,  0.2320,  0.3619,  ...,  0.1845, -0.0433,  0.1728],
          [-0.3976,  0.2420, -0.2093,  ..., -0.3422, -0.0767,  0.0779],
          [-0.4428, -0.0187,  0.0433,  ..., -0.3104, -0.0829, -0.0877]],

         [[-0.0666,  0.0319, -0.0342,  ...,  0.0834,  0.1865,  0.1134],
          [ 0.3491, -0.0077, -0.1411,  ..., -0.1849,  0.0924, -0.0364],
          [ 0.1105, -0.0031,  0.0369,  ..., -0.5109,  0.0167, -0.1582],
          ...,
          [-0.0015, -0.3552, -0.0852,  ..., -0.0170,  0.1921, -0.1876],
          [-0.1492, -0.2240, -0.6237,  ...,  0.1862, -0.1836, -0.1919],
          [ 0.0779, -0.0264, -0.3977,  ...,  0.0978, -0.2226,  0.1559]]],


        [[[ 0.1326, -0.2341, -0.1868,  ..., -0.0324,  0.2228,  0.1212],
          [ 0.1068,  0.0321,

In [31]:
h_n.view(num_layers, num_dirs, batch_size, hidden_size).shape

torch.Size([2, 2, 10, 512])

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.

